## Model

In [1]:
import transformers

print(transformers.__version__)

4.39.0


In [29]:
import pandas as pd
import numpy as np
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import set_seed, AutoTokenizer, AutoModel, DistilBertTokenizer, DistilBertForSequenceClassification

from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
import datasets
import random
from IPython.display import display, HTML

In [4]:
#!pip install datasets

In [5]:
torch.cuda.is_available()

True

## Helper functions

In [6]:
class AbstractsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx] for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Note the dtype change for binary classification
        return item

    def __len__(self):
        return len(self.labels)

## Set seed

In [7]:
set_seed(6013)

## BioBERT

In [8]:
dat = pd.read_csv('zotero_data.csv')
dat['abstract'] = dat['abstract'].astype(str)

In [9]:
labels = dat['fus_related'].tolist()
print("Label distribution:", np.bincount(labels))

Label distribution: [1794 1794]


In [10]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
tokenized_data = tokenizer(list(dat['abstract']), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [11]:
dataset = AbstractsDataset(tokenized_data, labels)

In [12]:
# Split indices into train and test sets first
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=0.1, random_state=42)

# Split train indices into train and validation sets
train_idx, val_idx = train_test_split(train_idx, test_size=0.1, random_state=42)

# Create Subset datasets
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)
print(f"Dataset size - train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")

Dataset size - train: 2906, val: 323, test: 359


In [14]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=0.001, 
    report_to=None,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    logging_strategy="epoch",  # Log at the end of each epoch
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=1)  # Adjust zero_division here
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Hyper-param tuning

In [15]:
#! pip install ray[tune]

In [16]:
import ray
from ray import train, tune

In [17]:
ray.init()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Python version:,3.10.12
Ray version:,2.10.0


In [18]:
# Print the status of the Ray cluster.
print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
    {} Memory resources in total
    {} Object store memory resources in total
    '''.format(len(ray.nodes()), ray.cluster_resources()['CPU'], 
           ray.cluster_resources()['memory'] / (1024*1024*1024), 
           ray.cluster_resources()['object_store_memory'] / (1024*1024*1024)))

This cluster consists of
    1 nodes in total
    48.0 CPU resources in total
    135.0188413625583 Memory resources in total
    61.85660018865019 Object store memory resources in total
    


In [19]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=2)

In [20]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.create_optimizer_and_scheduler(num_training_steps=1000)
trainer.optimizer = torch.optim.AdamW(trainer.model.parameters(),lr=5e-5)
trainer.lr_scheduler = get_linear_schedule_with_warmup(trainer.optimizer, num_warmup_steps=500, num_training_steps=1000)

/home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:835: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tenso

In [21]:
def ray_hp_space(trial):
    return {
        "per_device_train_batch_size": tune.grid_search([8, 16, 32]),
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4])
    }

In [22]:
best_run = trainer.hyperparameter_search(n_trials=1, 
                                        direction="maximize",
                                        hp_space=ray_hp_space)

2024-04-09 13:09:09,671	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-04-09 13:09:10 (running for 00:00:00.47)
Using FIFO scheduling algorithm.
Logical resource usage: 0/48 CPUs, 0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-09_13-08-28_650867_97883/artifacts/2024-04-09_13-09-09/_objective_2024-04-09_13-09-09/driver_artifacts
Number of trials: 27/27 (27 PENDING)
+------------------------+----------+-------+-----------------+--------------------+------------------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_bat |
|                        |          |       |                 |                    |                ch_size |
|------------------------+----------+-------+-----------------+--------------------+------------------------|
| _objective_e1659_00000 | PENDING  |       |           2e-05 |                  2 |                      8 |
| _objective_e1659_00001 | PENDING  |       |           3e-05 |                  2 |     

(_objective pid=101249) /home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
(_objective pid=101249) dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
(_objective pid=101249)   warnings.warn(
(_objective pid=101249) Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
(_objective pid=101246) /usr/local/lib/python3.10/dist-packages/torch/_utils.py:835: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage

== Status ==
Current time: 2024-04-09 13:10:46 (running for 00:01:36.42)
Using FIFO scheduling algorithm.
Logical resource usage: 4.0/48 CPUs, 4.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-09_13-08-28_650867_97883/artifacts/2024-04-09_13-09-09/_objective_2024-04-09_13-09-09/driver_artifacts
Number of trials: 27/27 (23 PENDING, 4 RUNNING)
+------------------------+----------+---------------------+-----------------+--------------------+------------------------+
| Trial name             | status   | loc                 |   learning_rate |   num_train_epochs |   per_device_train_bat |
|                        |          |                     |                 |                    |                ch_size |
|------------------------+----------+---------------------+-----------------+--------------------+------------------------|
| _objective_e1659_00000 | RUNNING  | 10.155.48.67:101246 |           2e-05 |                  2 |                      8 |
| _o

(_objective pid=101246) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
(_objective pid=101246) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/728 [00:00<?, ?it/s]


...

(_objective pid=108574) {'loss': 0.2091, 'grad_norm': 1.5710190534591675, 'learning_rate': 3.6400000000000004e-05, 'epoch': 4.0}


(_objective pid=108574) 
 14%|█▍        | 3/21 [00:00<00:00, 19.12it/s]
(_objective pid=108574) 
 24%|██▍       | 5/21 [00:00<00:01, 15.63it/s]
(_objective pid=108574) 
 33%|███▎      | 7/21 [00:00<00:00, 14.53it/s]
(_objective pid=108574) 
 43%|████▎     | 9/21 [00:00<00:01,  9.89it/s]
(_objective pid=108574) 
 52%|█████▏    | 11/21 [00:00<00:00, 10.64it/s]
(_objective pid=108574) 
 62%|██████▏   | 13/21 [00:01<00:00, 11.22it/s]
(_objective pid=108574) 
 71%|███████▏  | 15/21 [00:01<00:00, 11.70it/s]
(_objective pid=108574) 
 81%|████████  | 17/21 [00:01<00:00, 12.04it/s]
(_objective pid=108574) 
 90%|█████████ | 19/21 [00:01<00:00, 12.34it/s]
                                                 
100%|██████████| 364/364 [03:15<00:00,  1.86it/s]A


(_objective pid=108574) {'eval_loss': 0.166115403175354, 'eval_accuracy': 0.9597523219814241, 'eval_f1': 0.9623188405797102, 'eval_precision': 0.9378531073446328, 'eval_recall': 0.9880952380952381, 'eval_runtime': 1.7297, 'eval_samples_per_second': 186.737, 'eval_steps_per_second': 12.141, 'epoch': 4.0}
(_objective pid=108574) {'train_runtime': 195.2189, 'train_samples_per_second': 59.543, 'train_steps_per_second': 1.865, 'train_loss': 0.3307206499707568, 'epoch': 4.0}


2024-04-09 13:53:56,620	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/home/re4mfy/ray_results/_objective_2024-04-09_13-09-09' in 0.2638s.
2024-04-09 13:53:56,638	INFO tune.py:1048 -- Total run time: 2686.97 seconds (2686.57 seconds for the tuning loop).


== Status ==
Current time: 2024-04-09 13:53:56 (running for 00:44:46.83)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/48 CPUs, 1.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-09_13-08-28_650867_97883/artifacts/2024-04-09_13-09-09/_objective_2024-04-09_13-09-09/driver_artifacts
Number of trials: 27/27 (27 TERMINATED)
+------------------------+------------+---------------------+-----------------+--------------------+------------------------+-------------+
| Trial name             | status     | loc                 |   learning_rate |   num_train_epochs |   per_device_train_bat |   objective |
|                        |            |                     |                 |                    |                ch_size |             |
|------------------------+------------+---------------------+-----------------+--------------------+------------------------+-------------|
| _objective_e1659_00000 | TERMINATED | 10.155.48.67:101246 |           

In [25]:
for n, v in best_run.hyperparameters.items():
    print(n, v)

per_device_train_batch_size 8
learning_rate 3e-05
num_train_epochs 4


In [23]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.423300,0.167216,0.953560,0.956772,0.927374,0.988095
2,0.261000,0.181490,0.947368,0.949853,0.941520,0.958333
3,0.245100,0.140653,0.962848,0.965517,0.933333,1.000000
4,0.223000,0.137267,0.965944,0.968300,0.938547,1.000000


TrainOutput(global_step=1456, training_loss=0.28808686497447256, metrics={'train_runtime': 210.9921, 'train_samples_per_second': 55.092, 'train_steps_per_second': 6.901, 'total_flos': 3058402907504640.0, 'train_loss': 0.28808686497447256, 'epoch': 4.0})

In [26]:
trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 0.31502318382263184,
 'eval_accuracy': 0.9108635097493036,
 'eval_f1': 0.9139784946236558,
 'eval_precision': 0.85,
 'eval_recall': 0.9883720930232558,
 'eval_runtime': 1.858,
 'eval_samples_per_second': 193.22,
 'eval_steps_per_second': 12.379,
 'epoch': 4.0}